In [8]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
lista=['XRPUSDT',	
'XTZUSDT',	
'OMGUSDT',	
'CRVUSDT',	
'1INCHUSDT',
'MANAUSDT',
'FLOWUSDT',
'MAGICUSDT',
'MINAUSDT',
'PHBUSDT',	
'PERPUSDT',
'IDUSDT',	
'JOEUSDT',	
'HFTUSDT',	
'EDUUSDT',	
'SUIUSDT',	
'COMBOUSDT',
'MAVUSDT'	]

#lista=md.lista_de_monedas ()
#lista=['1INCHUSDT']

for symbol in lista:    
    try:
        #data = md.sigo_variacion_bitcoin(symbol)
        #data = md.estrategia_bb(symbol)
        data = md.estrategia_santa(symbol,tp_flag=True)
        resultado = md.backtesting(data, plot_flag = True)
        #if (
        #    (resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor']))
        #    and (resultado['Return [%]']/resultado['# Trades']) >=0.33 
        #    ):
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        pass

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


1INCHUSDT - Return [%]: -0.75% - # Trades: 6 - Profit Factor: 0.69 - Win Rate [%]: 16.666666666666664


In [5]:
data = md.estrategia_santa('HFTUSDT',tp_flag=True)
resultado = md.backtesting(data, plot_flag = False)
resultado

Start                     2023-07-10 14:30:00
End                       2023-07-21 00:15:00
Duration                     10 days 09:45:00
Exposure Time [%]                         0.0
Equity Final [$]                       1000.0
Equity Peak [$]                        1000.0
Return [%]                                0.0
Buy & Hold Return [%]                6.136762
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    